<a href="https://colab.research.google.com/github/Mnikito/4016/blob/main/HW2/HW2_Problem2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import numpy as np 
import pandas as pd 

import torch
import torch.optim as optim
import torch.nn as nn

torch.set_printoptions(edgeitems=2, linewidth=75)
 
# Data Visualisation 
import matplotlib.pyplot as plt  
import seaborn as sns

torch.manual_seed(123)

In [2]:
#Download CIFAR10
from torchvision import datasets

cifar10 = datasets.CIFAR10('data', train=True, download=True)
cifar10_val = datasets.CIFAR10('data', train=False, download=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [3]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']
num_classes = 10

In [4]:
from torchvision import transforms


to_tensor = transforms.ToTensor()

In [5]:
cifar10 = datasets.CIFAR10('data', train=True, download=False,
                          transform=transforms.ToTensor())

imgs = torch.stack([img for img, _ in cifar10], dim=3)

In [6]:
imgs.view(3, -1).mean(dim=1)

tensor([0.4914, 0.4822, 0.4465])

In [7]:
imgs.view(3, -1).std(dim=1)

tensor([0.2470, 0.2435, 0.2616])

In [8]:
cifar10 = datasets.CIFAR10('data', train=True, download=False,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.4915, 0.4823, 0.4468),
                                                   (0.2470, 0.2435, 0.2616))
                          ]))

In [9]:
cifar10_val = datasets.CIFAR10('data', train=False, download=False,
                          transform=transforms.Compose([
                              transforms.ToTensor(),
                              transforms.Normalize((0.4915, 0.4823, 0.4468),
                                                   (0.2470, 0.2435, 0.2616))
                          ]))

In [10]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)

model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.Tanh(),
            nn.Linear(512, 10),
            nn.LogSoftmax(dim=1))

learning_rate = 1e-2

optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.NLLLoss()

n_epochs = 300

for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 1.839059
Epoch: 1, Loss: 1.377479
Epoch: 2, Loss: 1.992995
Epoch: 3, Loss: 1.299178
Epoch: 4, Loss: 1.646823
Epoch: 5, Loss: 1.176374
Epoch: 6, Loss: 1.235652
Epoch: 7, Loss: 1.330879
Epoch: 8, Loss: 2.059225
Epoch: 9, Loss: 1.036934
Epoch: 10, Loss: 1.231704
Epoch: 11, Loss: 1.329209
Epoch: 12, Loss: 1.224085
Epoch: 13, Loss: 1.485193
Epoch: 14, Loss: 0.891266
Epoch: 15, Loss: 1.168995
Epoch: 16, Loss: 1.122203
Epoch: 17, Loss: 1.354010
Epoch: 18, Loss: 0.666645
Epoch: 19, Loss: 1.155142
Epoch: 20, Loss: 1.205740
Epoch: 21, Loss: 1.132131
Epoch: 22, Loss: 1.020363
Epoch: 23, Loss: 0.985348
Epoch: 24, Loss: 0.934238
Epoch: 25, Loss: 1.079740
Epoch: 26, Loss: 0.973233
Epoch: 27, Loss: 0.695339
Epoch: 28, Loss: 0.756448
Epoch: 29, Loss: 0.756735
Epoch: 30, Loss: 1.070954
Epoch: 31, Loss: 0.660786
Epoch: 32, Loss: 0.645945
Epoch: 33, Loss: 0.703800
Epoch: 34, Loss: 0.493903
Epoch: 35, Loss: 0.348452
Epoch: 36, Loss: 0.575784
Epoch: 37, Loss: 0.558789
Epoch: 38, Loss: 0.642

In [11]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))

Accuracy: 1.000000


In [12]:
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        outputs = model(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %f" % (correct / total))

Accuracy: 0.467600
